# Caching with `gw-signal-tools`

How it works:



TODO: maybe just make disclaimer that caching has to be DISABLED
before any code is imported for the first time. By default it is on
-> other way around would also work
-> reason is that re-importing does not recognize that cache_func
   is something new now
Alternatively: define waveform wrapper manually with cache, leaving
it turnt off -> might be best, active option that can be disabled easily

I like following idea: by default, caching is DISABLED. One can ENABLE
it as first operation in script, then waveform generators returned by
gw_signal_tools functions will have it ENABLED natively.
Either way, one can always ENABLE it manually by defining waveform
generators with the cache wrapper (or any custom cache wrapper too,
for that matter). Main part that gw_signal_tools does is provide class
for dictionaries (waveform input) that can be cached.
Note that in all cases, dictionaries passed to the functions have to
be initialized as a HashableDict, there is now way of doing this
automatically (except for adding hash property do dict directly, but
this seems like it would be really bad coding practice)

In [1]:
import astropy.units as u
from gw_signal_tools import enable_caching, disable_caching
from gw_signal_tools.types import HashableDict


# Note: for changes in which of following lines is commented, the kernel
# has to be restarted entirely! Simply running the script again is not
# sufficient
enable_caching()  # Uncomment to enable caching
# disable_caching()  # To make sure caching is disabled
# If one of the imports here is changed, kernel has to be restarted

/home/user/miniconda3/envs/gw_signal_tools_light/lib/python3.11/site-packages/gwpy/time/__init__.py:36: UserWarning: Wswiglal-redir-stdio:

SWIGLAL standard output/error redirection is enabled in IPython.
This may lead to performance penalties. To disable locally, use:

with lal.no_swig_redirect_standard_output_error():
    ...

To disable globally, use:

lal.swig_redirect_standard_output_error(False)

Note however that this will likely lead to error messages from
LAL functions being either misdirected or lost when called from
Jupyter notebooks.

To suppress this warning, use:

import warnings
warnings.filterwarnings("ignore", "Wswiglal-redir-stdio")
import lal

  from lal import LIGOTimeGPS
2024-08-07  14:32:33  INFO (caching.py: 28): Enabling caching


In [2]:
from gw_signal_tools.waveform_utils import get_wf_generator

The pyseobnr package has failed to load, you will not be able to employ SEOBNRv5 approximants.


In [3]:
f_min = 20.*u.Hz
f_max = 1024.*u.Hz

wf_params = {
    'mass1': 50.*u.solMass,
    'mass2': 50*u.solMass,
    'deltaT': 1./2048.*u.s,
    'f22_start': f_min,
    'f_max': f_max,
    'f22_ref': 20.*u.Hz,
    'phi_ref': 0.*u.rad,
    'distance': 1.*u.Mpc,
    'inclination': 0.0*u.rad,
    'eccentricity': 0.*u.dimensionless_unscaled,
    'longAscNodes': 0.*u.rad,
    'meanPerAno': 0.*u.rad,
    'condition': 0
}

hashable_wf_params = HashableDict(wf_params)

In [4]:
from gw_signal_tools.waveform_utils import get_wf_generator
wf_gen = get_wf_generator('IMRPhenomXPHM')

/home/user/miniconda3/envs/gw_signal_tools_light/lib/python3.11/site-packages/lalsimulation/gwsignal/core/waveform.py:226: UserWarning: This code is currently UNREVIEWED, use with caution!
  warnings.warn("This code is currently UNREVIEWED, use with caution!")


In [5]:
from gw_signal_tools.caching import cache
wf_gen_with_caching = cache(get_wf_generator('IMRPhenomXPHM'))
# NOTE: any cacher can be used herer as wrapper

These calls always work

In [6]:
wf_gen_with_caching(hashable_wf_params)
wf_gen(hashable_wf_params)

<FrequencySeries([0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j,
                  0.+0.j]
                 unit=Unit("s"),
                 f0=<Quantity 0. Hz>,
                 df=<Quantity 0.0625 Hz>,
                 epoch=<Time object: scale='utc' format='gps' value=0.0>,
                 name='hplus',
                 channel=None)>

This call only works if caching has been enabled immediately at the beginning of the file

In [7]:
wf_gen(wf_params)

TypeError: unhashable type: 'dict'

Unfortunately, running disable here has no effect if it has been enabled at the beginning

In [8]:
disable_caching()

2024-08-07  14:32:39  INFO (caching.py: 19): Disabling caching


In [9]:
wf_gen(wf_params)

TypeError: unhashable type: 'dict'

Even new imports have no effect

In [10]:
from gw_signal_tools.waveform_utils import get_wf_generator
new_wf_gen = get_wf_generator('IMRPhenomXPHM')

In [11]:
new_wf_gen(wf_params)

TypeError: unhashable type: 'dict'